In [2]:
library(Seurat)
library(ggplot2)
library(mltools)
library(data.table)
library(lsa)
library(aricode)
library(kBET)
library(cowplot)
options(repr.plot.width=14, repr.plot.height=10)
library(viridis)
library(mclust)

seur <- readRDS('/gpfs/ysm/pi/zhao-data/wd262/new_cytof/write/BCR/5celltype/seur_patient1_BCR-XL.rds')
rst <- readRDS('/gpfs/ysm/pi/zhao-data/wd262/new_cytof/write/BCR/5celltype/rst_patient1_BCR-XL.rds')
truth <- seur$label

In [3]:
celltype_pred <- seur$pred
rownames(rst$H) <- rst$H[,1]
H_est_subsetting <- data.frame(rst$H[,-1])
H_est_subsetting <- apply(H_est_subsetting,2,function(x) x/sum(x))
celltypes <- c('CD4 T-cells', 'CD8 T-cells', 'monocytes', 'naïve B cells', 'NK cells')
H_est_subsetting <- H_est_subsetting[celltypes,]

In [5]:
cutoff.list <- c(1:100)/100                          
df <- data.frame(cutoff = cutoff.list)
df$acc <- df$N <- df$cos <- df$nmi <- df$ari <- df$sil <- 0   

In [7]:
j=1
cnt_max = 0
    N_subsetting = 0
    idx.subsetting = c()
        for(i in 1:length(truth)){
            if(max(H_est_subsetting[,i])>cutoff.list[j]){
                idx.subsetting = c(idx.subsetting,i)
                N_subsetting = N_subsetting+1
                cnt_max = cnt_max + 1*(truth[i]==celltype_pred[i])
            }

        }
    if(N_subsetting == 0) {
        df$acc[j] <- 0
    } else {
       df$acc[j]<-cnt_max/N_subsetting
    } 

In [8]:
df$acc

[1] 0.8829517 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
  [8] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [15] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [22] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [29] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [36] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [43] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [50] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [57] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [64] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [71] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [78] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [85] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [92] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [99] 0.0000000 0.0000000

In [9]:
df$N[j] <- N_subsetting
    
        truth_sub <- truth[idx.subsetting]
        truth_fact_sub <- as.factor(truth_sub)

        
        celltype_pred_sub <- seur$pred[idx.subsetting]
        celltype_pred_fact_sub <- as.factor(celltype_pred_sub)
        celltype_pred_fact_sub <- factor(celltype_pred_fact_sub, levels = levels(truth_fact_sub))

In [10]:
celltype_pred_fact_sub

1             2             3             4             5 
  CD4 T-cells   CD4 T-cells      NK cells   CD4 T-cells      NK cells 
            6             7             8             9            10 
  CD8 T-cells   CD8 T-cells      NK cells naïve B cells   CD8 T-cells 
           11            12            13            14            15 
  CD4 T-cells      NK cells   CD8 T-cells   CD4 T-cells      NK cells 
           16            17            18            19            20 
  CD8 T-cells   CD8 T-cells   CD8 T-cells   CD4 T-cells   CD8 T-cells 
           21            22            23            24            25 
     NK cells      NK cells      NK cells   CD8 T-cells   CD8 T-cells 
           26            27            28            29            30 
  CD4 T-cells   CD8 T-cells naïve B cells   CD8 T-cells      NK cells 
           31            32            33            34            35 
  CD8 T-cells   CD4 T-cells      NK cells   CD4 T-cells   CD4 T-cells 
           36            37            38            39            40 
  CD8 T-cells   CD8 T-cells      NK cells   CD4 T-cells   CD8 T-cells 
           41            42            43            44            45 
  CD4 T-cells   CD8 T-cells     monocytes   CD4 T-cells   CD4 T-cells 
           46            47            48            49            50 
  CD8 T-cells   CD8 T-cells      NK cells naïve B cells   CD4 T-cells 
           51            52            53            54            55 
  CD4 T-cells   CD8 T-cells   CD8 T-cells   CD4 T-cells   CD8 T-cells 
           56            57            58            59            60 
naïve B cells      NK cells   CD8 T-cells      NK cells   CD4 T-cells 
           61            62            63            64            65 
     NK cells   CD8 T-cells   CD4 T-cells   CD8 T-cells   CD8 T-cells 
           66            67            68            69            70 
  CD8 T-cells   CD8 T-cells naïve B cells      NK cells   CD8 T-cells 
           71            72            73            74            75 
     NK cells   CD8 T-cells   CD8 T-cells   CD8 T-cells      NK cells 
           76            77            78            79            80 
  CD8 T-cells      NK cells      NK cells   CD4 T-cells   CD8 T-cells 
           81            82            83            84            85 
  CD8 T-cells   CD8 T-cells   CD8 T-cells   CD8 T-cells naïve B cells 
           86            87            88            89            90 
  CD4 T-cells   CD8 T-cells   CD4 T-cells     monocytes   CD4 T-cells 
           91            92            93            94            95 
  CD4 T-cells      NK cells   CD4 T-cells      NK cells   CD8 T-cells 
           96            97            98            99           100 
  CD4 T-cells   CD4 T-cells   CD8 T-cells      NK cells      NK cells 
          101           102           103           104           105 
  CD4 T-cells      NK cells   CD8 T-cells   CD4 T-cells   CD4 T-cells 
          106           107           108           109           110 
     NK cells      NK cells      NK cells      NK cells      NK cells 
          111           112           113           114           115 
  CD8 T-cells   CD8 T-cells   CD4 T-cells   CD4 T-cells   CD8 T-cells 
          116           117           118           119           120 
     NK cells naïve B cells   CD8 T-cells   CD8 T-cells      NK cells 
          121           122           123           124           125 
  CD8 T-cells   CD4 T-cells   CD8 T-cells   CD8 T-cells   CD4 T-cells 
          126           127           128           129           130 
  CD4 T-cells   CD4 T-cells   CD8 T-cells   CD8 T-cells   CD8 T-cells 
          131           132           133           134           135 
naïve B cells   CD8 T-cells   CD8 T-cells   CD8 T-cells   CD4 T-cells 
          136           137           138           139           140 
  CD8 T-cells   CD8 T-cells   CD4 T-cells   CD8 T-cells   CD4 T-cells 
          141           143         

In [12]:
length(unique(celltype_pred_sub))

[1] 5

In [13]:
pca.data <- list()
            pca.data$x <- seur@reductions$pca@cell.embeddings[idx.subsetting,]
            sil <- batch_sil(pca.data, as.numeric(as.factor(celltype_pred_sub)))
            df$sil[j] <- sil

In [14]:
df$sil

[1] 0.1072708 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
  [8] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [15] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [22] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [29] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [36] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [43] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [50] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [57] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [64] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [71] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [78] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [85] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [92] 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000 0.0000000
 [99] 0.0000000 0.0000000

In [15]:
# for(j in c(1:length(cutoff.list))){
for(j in ){
    ## cnt_max
    cnt_max = 0
    N_subsetting = 0
    idx.subsetting = c()
        for(i in 1:length(truth)){
            if(max(H_est_subsetting[,i])>cutoff.list[j]){
                idx.subsetting = c(idx.subsetting,i)
                N_subsetting = N_subsetting+1
                cnt_max = cnt_max + 1*(truth[i]==celltype_pred[i])
            }

        }
    if(N_subsetting == 0) {
        df$acc[j] <- 0
    } else {
       df$acc[j]<-cnt_max/N_subsetting
    } 
    if(length(idx.subsetting) > 0) {
        df$N[j] <- N_subsetting
    
        truth_sub <- truth[idx.subsetting]
        truth_fact_sub <- as.factor(truth_sub)

        
        celltype_pred_sub <- seur$pred[idx.subsetting]
        celltype_pred_fact_sub <- as.factor(celltype_pred_sub)
        celltype_pred_fact_sub <- factor(celltype_pred_fact_sub, levels = levels(truth_fact_sub))

        ## ARI
        ari <- adjustedRandIndex(celltype_pred_sub, truth_sub)
        df$ari[j] <- ari
        
        ## Cosine similarity
        truth_onehot_sub <- as.data.frame(t(one_hot(as.data.table(truth_fact_sub))))
        cos_sim <- mean(mapply(cosine, truth_onehot_sub, as.data.frame(H_est_subsetting[,idx.subsetting])))
        df$cos[j] <- cos_sim

        ## NMI
        pred_onehot_sub <- as.data.frame(t(one_hot(as.data.table(celltype_pred_fact_sub))))
        nmi <- mean(mapply(NMI, truth_onehot_sub, as.data.frame(pred_onehot_sub)))
        df$nmi[j] <- nmi

        ## Silhouette
        if(length(unique(celltype_pred_sub) == 1)) {
            df$sil[j] <- 0
        } else{
            pca.data <- list()
            pca.data$x <- seur@reductions$pca@cell.embeddings[idx.subsetting,]
            sil <- batch_sil(pca.data, as.numeric(as.factor(celltype_pred_sub)))
            df$sil[j] <- sil
        }   
    }
}

In [16]:
df

cutoff,sil,ari,nmi,cos,N,acc
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.01,0,0.7129697,0.8947721,0.8739348,2751,0.8829517
0.02,0,0.7129697,0.8947721,0.8739348,2751,0.8829517
0.03,0,0.7129697,0.8947721,0.8739348,2751,0.8829517
0.04,0,0.7129697,0.8947721,0.8739348,2751,0.8829517
0.05,0,0.7129697,0.8947721,0.8739348,2751,0.8829517
0.06,0,0.7129697,0.8947721,0.8739348,2751,0.8829517
0.07,0,0.7129697,0.8947721,0.8739348,2751,0.8829517
0.08,0,0.7129697,0.8947721,0.8739348,2751,0.8829517
0.09,0,0.7129697,0.8947721,0.8739348,2751,0.8829517
